In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [26]:
d="""COD,STATE,CITY,AZIM,SET,TEC
ALAAD_0001,AL,MAC,0,1,4
ALAAD_0001,AL,MAC,120,2,4
ALAAD_0001,AL,MAC,120,2,4
ALAAD_0001,AL,MAC,240,3,4
BAPID_0001,BA,SAL,20,1,2
BAPID_0001,BA,SAL,20,1,2
BAPID_0001,BA,SAL,100,2,2
BAPID_0001,BA,SAL,210,3,2
BAPID_0001,BA,SAL,250,3,2
BAPID_0001,BA,SAL,250,3,2
CEMBC_0003,CE,FOR,90,1,4
CEMBC_0003,CE,FOR,80,1,4
CEMBC_0003,CE,FOR,160,2,4
CEMBC_0003,CE,FOR,160,2,4
CEMBC_0003,CE,FOR,170,2,4
CEMBC_0003,CE,FOR,280,3,4"""
df=g(d)
df

,COD,STATE,CITY,AZIM,SET,TEC
0,ALAAD_0001,AL,MAC,0,1,4
1,ALAAD_0001,AL,MAC,120,2,4
2,ALAAD_0001,AL,MAC,120,2,4
3,ALAAD_0001,AL,MAC,240,3,4
4,BAPID_0001,BA,SAL,20,1,2
5,BAPID_0001,BA,SAL,20,1,2
6,BAPID_0001,BA,SAL,100,2,2
7,BAPID_0001,BA,SAL,210,3,2
8,BAPID_0001,BA,SAL,250,3,2
9,BAPID_0001,BA,SAL,250,3,2


In [27]:
df_cut = (
    df.groupby(["COD","TEC","SET"])
    .AZIM
    .agg(lambda x: pd.Series.mode(x).max())
    .reset_index()
)
df_cut

,COD,TEC,SET,AZIM
0,ALAAD_0001,4,1,0
1,ALAAD_0001,4,2,120
2,ALAAD_0001,4,3,240
3,BAPID_0001,2,1,20
4,BAPID_0001,2,2,100
5,BAPID_0001,2,3,250
6,CEMBC_0003,4,1,90
7,CEMBC_0003,4,2,160
8,CEMBC_0003,4,3,280


In [30]:
df_cut.merge(df.drop_duplicates(), on=["COD","TEC","SET", "AZIM"])

,COD,TEC,SET,AZIM,STATE,CITY
0,ALAAD_0001,4,1,0,AL,MAC
1,ALAAD_0001,4,2,120,AL,MAC
2,ALAAD_0001,4,3,240,AL,MAC
3,BAPID_0001,2,1,20,BA,SAL
4,BAPID_0001,2,2,100,BA,SAL
5,BAPID_0001,2,3,250,BA,SAL
6,CEMBC_0003,4,1,90,CE,FOR
7,CEMBC_0003,4,2,160,CE,FOR
8,CEMBC_0003,4,3,280,CE,FOR


In [16]:
df_cut.combine_first(df)

,AZIM,CITY,COD,SET,STATE,TEC
0,0.0,MAC,ALAAD_0001,1.0,AL,4.0
1,120.0,MAC,ALAAD_0001,2.0,AL,4.0
2,240.0,MAC,ALAAD_0001,3.0,AL,4.0
3,20.0,MAC,BAPID_0001,1.0,AL,2.0
4,100.0,SAL,BAPID_0001,2.0,BA,2.0
5,250.0,SAL,BAPID_0001,3.0,BA,2.0
6,90.0,SAL,CEMBC_0003,1.0,BA,4.0
7,160.0,SAL,CEMBC_0003,2.0,BA,4.0
8,280.0,SAL,CEMBC_0003,3.0,BA,4.0
9,250.0,SAL,BAPID_0001,3.0,BA,2.0


In [53]:
idx = [1, 2, 3]
colA = [10, 5, 12]
n = [3, 2, 4]
x1 = [0, 1, 7]
y1 = [4, 0, 4]
x2 = [3, 2, 8]
y2 = [5, 1, 5]
x3 = [4, np.nan, 10]
y3 = [3,np.nan, 3]
x4 = [np.nan, np.nan, 11]
y4 = [np.nan, np.nan, 3]

df = pd.DataFrame(list(zip(idx, colA, n, 
                           x1, y1, x2, y2, x3, y3, x4, y4
                          )), 
                  columns =['idx', 'colA', 'n', 
                            'x1', 'y1', 'x2', 'y2', 
                            'x3', 'y3', 'x4', 'y4'
                           ])
df = df.drop(columns=['n'])
df

,idx,colA,x1,y1,x2,y2,x3,y3,x4,y4
0,1,10,0,4,3,5,4.0,3.0,NaN,NaN
1,2,5,1,0,2,1,NaN,NaN,NaN,NaN
2,3,12,7,4,8,5,10.0,3.0,11.0,3.0


In [57]:
#df.melt(id_vars=(['idx', 'colA'])).dropna()

In [76]:
d="""date_created
2018-12-31 23:30:20+00:00
2018-12-31 23:30:00+00:00
2018-12-31 23:28:09+00:00
2018-12-31 23:28:08+00:00"""
df=pd.read_csv(StringIO(d))
df

,date_created
0,2018-12-31 23:30:20+00:00
1,2018-12-31 23:30:00+00:00
2,2018-12-31 23:28:09+00:00
3,2018-12-31 23:28:08+00:00


In [77]:
df['date_created'] = pd.to_datetime(df.date_created,errors='coerce').dt.tz_localize(None)
df

,date_created
0,2018-12-31 23:30:20
1,2018-12-31 23:30:00
2,2018-12-31 23:28:09
3,2018-12-31 23:28:08


In [78]:
d="""col1,col2,col3,col4,col5,col6,col7
1,test1,t1,Dummy1,result1,10,100.2
2,test1,t1,Dummy2,result2,20,101.2
3,test1,t1,Dummy3,result3,30,102.3
4,test1,t1,Dummy4,result4,40,101.4
5,test2,t1,Dummy1,result1,10,101
6,test2,t1,Dummy2,result2,20,103
7,test2,t1,Dummy3,result3,30,104
8,test2,t1,Dummy4,result4,40,105
9,test3,t1,Dummy1,result1,10,102
10,test3,t1,Dummy2,result2,20,106
11,test3,t1,Dummy3,result3,30,107
12,test3,t1,Dummy5,result4,50,110.2
13,test4,t1,Dummy2,result2,20,120
14,test5,t1,Dummy6,result1,100,88"""
df=g(d)
df

,col1,col2,col3,col4,col5,col6,col7
0,1,test1,t1,Dummy1,result1,10,100.2
1,2,test1,t1,Dummy2,result2,20,101.2
2,3,test1,t1,Dummy3,result3,30,102.3
3,4,test1,t1,Dummy4,result4,40,101.4
4,5,test2,t1,Dummy1,result1,10,101.0
5,6,test2,t1,Dummy2,result2,20,103.0
6,7,test2,t1,Dummy3,result3,30,104.0
7,8,test2,t1,Dummy4,result4,40,105.0
8,9,test3,t1,Dummy1,result1,10,102.0
9,10,test3,t1,Dummy2,result2,20,106.0


In [103]:
d="""A,B,C,D
a,OK,one,col1
b,OK,two,col1
c,OK,two,col2
d,OK,four,NaN
e,OK,five,NaN
f,OK,six,NaN
g,NaN,NaN,col3
h,NaN,NaN,col4
i,NaN,NaN,col5"""
df=g(d)
df

,A,B,C,D
0,a,OK,one,col1
1,b,OK,two,col1
2,c,OK,two,col2
3,d,OK,four,NaN
4,e,OK,five,NaN
5,f,OK,six,NaN
6,g,NaN,NaN,col3
7,h,NaN,NaN,col4
8,i,NaN,NaN,col5


In [104]:
df[['C','D']] = df[['C','D']].fillna('NA') # To keep things simple while dropping col
df.pivot(index='C', columns = 'D', 
         values = 'B').rename_axis(index=None, columns=None).drop(columns='NA', index='NA')

,col1,col2,col3,col4,col5
five,NaN,NaN,NaN,NaN,NaN
four,NaN,NaN,NaN,NaN,NaN
one,OK,NaN,NaN,NaN,NaN
six,NaN,NaN,NaN,NaN,NaN
two,OK,OK,NaN,NaN,NaN


In [108]:
l = [10, 20, 21, None, 32, 'asc']
ob=[True if i is None else i for i in l]
ob=[True if i is 'asc' else i for i in l]
ob

<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-108-b8674ded166a>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  ob=[True if i is 'asc' else i for i in l]


[10, 20, 21, None, 32, True]

In [118]:
value = 'Ramesh ,  SUresh , Chandu'
value = (value.split(','))
value = [x.strip() for x in value]
value

['Ramesh', 'SUresh', 'Chandu']

In [117]:
val = 'Ramesh ,  SUresh , Chandu'
val = [x.strip()for x in value.split(',')]

In [119]:
val == value

True

In [120]:
a,b = val[0:2]

In [122]:
b

'SUresh'

In [161]:
d="""A,B,C,D
3957,OK,One,TM-009.4
3957,OK,two,TM-009.4
4147,OK,three,CERT008
3816,OK,four,FITEYE-04
3955,OK,five,TM-009.2
4147,OK,six,CERT008
4147,OK,seven,CERT008
3807,OK,seven,EMT-038.4
nan,OK,eight,nan
nan,OK,nine,nan
nan,OK,ten,nan
nan,OK,11,nan
nan,OK,12,nan
nan,OK,13,nan
nan,OK,14,nan
nan,OK,14,nan
3814,nan,nan,FITEYE-02
3819,nan,nan,FITEYE-08
3884,nan,nan,TG-000.8
4087,nan,nan,TM-042.1"""
df=g(d)
df

,A,B,C,D
0,3957.0,OK,One,TM-009.4
1,3957.0,OK,two,TM-009.4
2,4147.0,OK,three,CERT008
3,3816.0,OK,four,FITEYE-04
4,3955.0,OK,five,TM-009.2
5,4147.0,OK,six,CERT008
6,4147.0,OK,seven,CERT008
7,3807.0,OK,seven,EMT-038.4
8,NaN,OK,eight,NaN
9,NaN,OK,nine,NaN


In [162]:
df[['C','D']] = df[['C','D']].fillna('NA')
df = df.drop_duplicates(['C'])
df.pivot(index = 'C', columns = 'D', values='B').rename_axis(index=None, columns=None).drop(columns='NA', index='NA')


,CERT008,FITEYE-02,FITEYE-04,TM-009.2,TM-009.4
11,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN
One,NaN,NaN,NaN,NaN,OK
eight,NaN,NaN,NaN,NaN,NaN
five,NaN,NaN,NaN,OK,NaN
four,NaN,NaN,OK,NaN,NaN
nine,NaN,NaN,NaN,NaN,NaN
seven,OK,NaN,NaN,NaN,NaN


In [164]:
d="""Country,ID,March,April,May,June
Netherlands,A,10,20,15,14
Netherlands,B,15,18,10,4
Germany,C,9,8,3,1"""
df1=g(d)
df1

,Country,ID,March,April,May,June
0,Netherlands,A,10,20,15,14
1,Netherlands,B,15,18,10,4
2,Germany,C,9,8,3,1


In [165]:
d="""Country,ID,Month,Number
Netherlands,A,March,8
Netherlands,A,March,28
Netherlands,B,April,18
Germany,C,Jan,9"""
df2 = g(d)
df2

,Country,ID,Month,Number
0,Netherlands,A,March,8
1,Netherlands,A,March,28
2,Netherlands,B,April,18
3,Germany,C,Jan,9


In [188]:
df1.merge(df2.pivot_table(
            index=['Country', 'ID'],
            columns='Month',
            values='Number',
            aggfunc=sum,
            fill_value=0).add_prefix(
        'df2_').reset_index(), on=['Country', 'ID'])

,Country,ID,March,April,May,June,df2_April,df2_Jan,df2_March
0,Netherlands,A,10,20,15,14,0,0,36
1,Netherlands,B,15,18,10,4,18,0,0
2,Germany,C,9,8,3,1,0,9,0


In [171]:
df1.merge(df3, on)

,Country,ID,March,April,May,June,Jan


In [128]:
d="""Identificator,Date,Status
ID1,2021-05-02 19:55:43,OK
ID2,2021-05-02 19:48:01,FAILED
ID3,2021-05-02 19:47:53,OK
ID1,2021-05-03 19:55:43,OK
ID2,2021-05-03 20:48:01,OK
ID1,2021-05-04 19:55:43,FAILED
ID1,2021-05-04 20:55:43,OK
ID2,2021-05-04 19:48:01,OK
ID3,2021-05-04 19:47:53,OK"""
df=g(d)
df

,Identificator,Date,Status
0,ID1,2021-05-02 19:55:43,OK
1,ID2,2021-05-02 19:48:01,FAILED
2,ID3,2021-05-02 19:47:53,OK
3,ID1,2021-05-03 19:55:43,OK
4,ID2,2021-05-03 20:48:01,OK
5,ID1,2021-05-04 19:55:43,FAILED
6,ID1,2021-05-04 20:55:43,OK
7,ID2,2021-05-04 19:48:01,OK
8,ID3,2021-05-04 19:47:53,OK


In [129]:
df.Date = pd.to_datetime(df.Date)
#df = df.set_index('Date')
#df = df.sort_values('Identificator')
#df

In [125]:
s = pd.crosstab(df.Identificator,df.Date.dt.date).stack()
#df = df.append(s[s==0].reset_index().rename(columns={0:'Status'}).replace(0,'MISSED'))
#df

,Identificator,Date,Status
0,ID1,2021-05-02 19:55:43,OK
1,ID2,2021-05-02 19:48:01,FAILED
2,ID3,2021-05-02 19:47:53,OK
3,ID1,2021-05-03 19:55:43,OK
4,ID2,2021-05-03 20:48:01,OK
5,ID1,2021-05-04 19:55:43,FAILED
6,ID1,2021-05-04 20:55:43,OK
7,ID2,2021-05-04 19:48:01,OK
8,ID3,2021-05-04 19:47:53,OK
0,ID3,2021-05-03,MISSED


In [131]:
pd.crosstab(df.Identificator,df.Date.dt.date).stack()

Identificator  Date      
ID1            2021-05-02    1
               2021-05-03    1
               2021-05-04    2
ID2            2021-05-02    1
               2021-05-03    1
               2021-05-04    1
ID3            2021-05-02    1
               2021-05-03    0
               2021-05-04    1
dtype: int64

In [76]:
df['new_treat']= df.groupby(['id'])['treat'].shift(-1)
df['new_treat'] = df.groupby('id')['new_treat'].transform('first').astype(int)
df

,id,year,treat,new_treat
0,1,2008,0,2009
1,1,2009,2009,2009
2,1,2010,2010,2009
3,1,2011,0,2009
4,2,2008,0,0
5,2,2009,0,0
6,2,2010,0,0
7,2,2011,0,0


In [74]:
df['new_treat']= df.groupby(['id'])['treat'].shift(-1)
df['new_treat'] = df.groupby('id')['new_treat'].transform('cummin')
df

,id,year,treat,new_treat
0,1,2008,0,2009.0
1,1,2009,2009,2009.0
2,1,2010,2010,0.0
3,1,2011,0,NaN
4,2,2008,0,0.0
5,2,2009,0,0.0
6,2,2010,0,0.0
7,2,2011,0,NaN


In [77]:
df11 = pd.DataFrame({"code1": ["A123; A321; B478; B678; C567", "A321; A821; B448; B698; C577"], "code2": ["A; B5; N5; R4; H5", "A3; B; N; R7; H2"],"code3": ["A445; A323; A323; A445; A659", "A328; A328; A621; A442; A621"]},      index=[0, 1], )
df11

,code1,code2,code3
0,A123; A321; B478; B678; C567,A; B5; N5; R4; H5,A445; A323; A323; A445; A659
1,A321; A821; B448; B698; C577,A3; B; N; R7; H2,A328; A328; A621; A442; A621


In [134]:
d="""P[kW]
0
0
0
8
8
0"""
a=g(d)
a

,P[kW]
0,0
1,0
2,0
3,8
4,8
5,0


In [147]:
d="""P1[kW],P2[kW],Pn[kW]
2,2,2
3,3,3
3,3,3
4,4,4
2,2,2
2,2,2"""
b=g(d)
b

,P1[kW],P2[kW],Pn[kW]
0,2,2,2
1,3,3,3
2,3,3,3
3,4,4,4
4,2,2,2
5,2,2,2


In [148]:
b.iloc[:, 1] += a.iloc[:, 0]
b

,P1[kW],P2[kW],Pn[kW]
0,2,2,2
1,3,3,3
2,3,3,3
3,4,12,4
4,2,10,2
5,2,2,2


In [265]:
load = [0,0,0,0,0,0,0]
data = pd.DataFrame(load)
for i in range(1, 10):
    data[i] = data[0]
data

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0


In [266]:
overlay = pd.DataFrame([0,0,0,0,6,6,0])
overlay

,0
0,0
1,0
2,0
3,0
4,6
5,6
6,0


In [268]:
cols=[1,2,4,5,7,8]
data[cols] = data.iloc[:,cols] + overlay.iloc[:].values
data

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,6,6,0,6,6,0,6,6,0
5,0,6,6,0,6,6,0,6,6,0
6,0,0,0,0,0,0,0,0,0,0


In [248]:
pd.concat([data, nd], axis=1)

,0,1,2,3,4,5,6,7,8,9,1,2,4,5,7,8
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,6,6,6,6,6,6
5,0,0,0,0,0,0,0,0,0,0,6,6,6,6,6,6
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [259]:
cols=[1,2,4,5,7,8]
data[cols] = data[cols] + overlay.values
data

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,6,6,0,6,6,0,6,6,0
5,0,6,6,0,6,6,0,6,6,0
6,0,0,0,0,0,0,0,0,0,0


In [269]:
d = [
    {'2020-12-31 00:00:00': [123, 456, 789, 321]}, 
    {'2021-05-06 00:00:00': [999, 888, 777, 666]}
]
d

[{'2020-12-31 00:00:00': [123, 456, 789, 321]},
 {'2021-05-06 00:00:00': [999, 888, 777, 666]}]

In [274]:
pd.DataFrame(d).stack().apply(pd.Series).add_prefix('Value')

,,Value0,Value1,Value2,Value3
0,2020-12-31 00:00:00,123,456,789,321
1,2021-05-06 00:00:00,999,888,777,666


In [293]:
pd.DataFrame(d).stack().apply(
    pd.Series).add_prefix(
    'Value').reset_index().rename(columns={'level_0':'index', 'level_1':'Date'})

,level_0,Date,Value0,Value1,Value2,Value3
0,0,2020-12-31 00:00:00,123,456,789,321
1,1,2021-05-06 00:00:00,999,888,777,666


In [307]:
pd.DataFrame(d).stack().apply(pd.Series)


,,0,1,2,3
0,2020-12-31 00:00:00,123,456,789,321
1,2021-05-06 00:00:00,999,888,777,666


In [350]:
df = pd.DataFrame(data={
'dep': ["NYC", "PAR", "MAD"],
'arr': ["PAR", "SYD", "BCN"],
'ID': ["idx1", "idx1", "idx2"],
'step':[1, 2, 1]})
df

,dep,arr,ID,step
0,NYC,PAR,idx1,1
1,PAR,SYD,idx1,2
2,MAD,BCN,idx2,1


In [346]:
df = df.groupby('ID').apply(lambda x: x.sort_values('step')).reset_index(drop=True)
df

,dep,arr,ID,step
0,NYC,PAR,idx1,1
1,PAR,SYD,idx1,2
2,MAD,BCN,idx2,1


In [354]:
df.merge(df.groupby('ID').apply(lambda x: '-'.join(np.unique(
    x[['dep', 'arr']].values.flatten()))).reset_index(name='itinerary'), on='ID')

,dep,arr,ID,step,itinerary
0,NYC,PAR,idx1,1,NYC-PAR-SYD
1,PAR,SYD,idx1,2,NYC-PAR-SYD
2,MAD,BCN,idx2,1,BCN-MAD


In [373]:
d="""date,k1-v1,k1-v2,k2-v1,k2-v2,k1k3-v1,k1k3-v2,k4-v1,k4-v2
2021-01-05,2.0,7.0,NaN,NaN,NaN,NaN,9.0,6.0
2021-01-31,NaN,NaN,8.0,5.0,NaN,NaN,7.0,6.0
2021-02-15,9.0,5.0,NaN,3.0,4.0,NaN,NaN,NaN
2021-02-28,NaN,9.0,0.0,1.0,NaN,NaN,8.0,8.0
2021-03-20,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-31,NaN,NaN,8.0,NaN,3.0,NaN,8.0,0.0
2021-04-10,NaN,NaN,7.0,6.0,NaN,NaN,NaN,9.0
2021-04-30,NaN,6.0,NaN,NaN,NaN,NaN,1.0,NaN
2021-05-14,8.0,NaN,3.0,3.0,4.0,NaN,NaN,NaN
2021-05-31,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN"""
df=g(d)
df

,date,k1-v1,k1-v2,k2-v1,k2-v2,k1k3-v1,k1k3-v2,k4-v1,k4-v2
0,2021-01-05,2.0,7.0,NaN,NaN,NaN,NaN,9.0,6.0
1,2021-01-31,NaN,NaN,8.0,5.0,NaN,NaN,7.0,6.0
2,2021-02-15,9.0,5.0,NaN,3.0,4.0,NaN,NaN,NaN
3,2021-02-28,NaN,9.0,0.0,1.0,NaN,NaN,8.0,8.0
4,2021-03-20,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-03-31,NaN,NaN,8.0,NaN,3.0,NaN,8.0,0.0
6,2021-04-10,NaN,NaN,7.0,6.0,NaN,NaN,NaN,9.0
7,2021-04-30,NaN,6.0,NaN,NaN,NaN,NaN,1.0,NaN
8,2021-05-14,8.0,NaN,3.0,3.0,4.0,NaN,NaN,NaN
9,2021-05-31,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN


In [375]:
pd.wide_to_long(
        df.rename(columns=(lambda x: ''.join(x.split('-')[::-1]))),
        stubnames=['v2', 'v1'],
        i='date',
        j='keys',
        suffix='.*'
    ).stack(
).reset_index(0).groupby(level=[0, 1]).agg({'date': 'last'}).unstack(-1)

date            
              v2          v1
keys                        
k1    2021-04-30  2021-05-14
k1k3         NaN  2021-05-14
k2    2021-05-31  2021-05-31
k4    2021-04-10  2021-04-30

In [381]:
pd.wide_to_long(
        df.rename(columns=(lambda x: ''.join(x.split('-')[::-1]))),
        stubnames=['v2', 'v1'],
        i='date',
        j='keys',
        suffix='.*'
    )

,,v2,v1
date,keys,,
2021-01-05,k1,7.0,2.0
2021-01-31,k1,NaN,NaN
2021-02-15,k1,5.0,9.0
2021-02-28,k1,9.0,NaN
2021-03-20,k1,NaN,7.0
2021-03-31,k1,NaN,NaN
2021-04-10,k1,NaN,NaN
2021-04-30,k1,6.0,NaN
2021-05-14,k1,NaN,8.0


In [55]:
fd={0:0, 1:1}

def fib(n, fd):
    if not n in fd:
        fd[n] = fib(n-2, fd) + fib(n-1, fd)
    return fd[n]


In [56]:
%timeit fib(50, fd)

156 ns ± 1.72 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [57]:
%timeit fib(50, fd)

157 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [60]:
from functools import lru_cache
@lru_cache
def fib(n):
    if n in [0,1]:
        return n
    else:
        return fib(n-1) + fib(n-2)

In [64]:
fib(6)

8

In [73]:
## Binet's formula to find fib
def fib(n):
    iter_n = np.arange(n)
    sqrt_five = np.sqrt(5)
    alpha = (1 + sqrt_five) / 2
    beta = (1 - sqrt_five) / 2
    
    # np.rint is used to convert from float to int
    #return np.rint(((alpha ** iter_n) - (beta ** iter_n)) / sqrt_five)
    return (((alpha ** iter_n) - (beta ** iter_n)) / sqrt_five).astype(int)

In [79]:
## Binet's formula concise way
def fib(n):
    n = np.arange(n)
    sqrt_five = np.sqrt(5)
    alpha = (1 + sqrt_five) / 2
    return np.rint((alpha**n - (-1/alpha)**n)/sqrt_five)

In [80]:
fib(5)

array([0., 1., 1., 2., 3.])

In [88]:
from functools import reduce
reduce(lambda x,y: x+y, np.arange(2))

1

<enum 'Month'>